# IR PROJECT CODE

Members:

*   Omer
*   Lucas
*   Muhamad

# Download and Inspect the Collection

The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.

Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891

GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file

GEMMA3 270M SETUP

Based on: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(270M).ipynb#scrollTo=pCqnaKmlO1U9

In [ ]:
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install pyterrier
!pip install pyterrier[java]
!pip install gensim
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
!pip install nltk
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

from unsloth import FastLanguageModel
import torch
import torch.nn.functional as F
import pyterrier as pt
from pyterrier.measures import P, MAP, R, nDCG
from pyterrier_t5 import MonoT5ReRanker
import os
import json
import pandas as pd
import shutil
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import numpy as np
import pickle
from gensim.models import KeyedVectors

import csv
import xml.etree.ElementTree as ET
from xml.dom import minidom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2026.1.2 requ

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true" -o test.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true" -o train.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true" -o validation.json

import json

files = ["train.json", "validation.json", "test.json"]

for path in files:
    print(f"\n===== {path} =====")
    try:
        with open(path, "r", encoding="utf-8") as f:
            # Read a few hundred characters to see what kind of JSON it is
            head = f.read(500)
            print("Preview of first 500 characters:\n")
            print(head[:500])
        # Try to load only part of the file
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"\nLoaded {len(data)} items (list).")
            print("Dictionary keys:", list(data[0].keys()))
            print(json.dumps(data[0], indent=2)[:600])
        elif isinstance(data, dict):
            print("\nTop-level is a dictionary. Keys:", list(data.keys()))
            for k, v in data.items():
                if isinstance(v, list):
                    print(f"Key '{k}' contains a list of {len(v)} items.")
                    if v:
                        print("First item keys:", list(v[0].keys()))
                        print(json.dumps(v[0], indent=2)[:600])
                        break
        else:
            print(f"Unexpected top-level type: {type(data)}")
    except Exception as e:
        print(f"Could not parse {path} as JSON: {e}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1075  100  1075    0     0   2714      0 --:--:-- --:--:-- --:--:--  2721
100 71.5M  100 71.5M    0     0  31.0M      0  0:00:02  0:00:02 --:--:-- 47.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1085  100  1085    0     0   2978      0 --:--:-- --:--:-- --:--:--  2972
100 1315M  100 1315M    0     0  68.5M      0  0:00:19  0:00:19 --:--:-- 97.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1079  100  1079    0     0   3512      0 --:--:-- --:--:-- --:--:--  3503
100 71.8M  100 71.8M    0     0  35.2M      0  0:00:02  0:00:02 --:--:-- 97.8M

===== train.json =====
Preview of first 500 charact

# Create the Document Collection

To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.

para_id: is the id of a paragraph of a news paper page.

In [ ]:
import json
import os

inputs = ["train.json", "validation.json", "test.json"]
output = "document_collection.json"

def load_list_or_empty(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Skipping {path} because it is missing or empty")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        print(f"Skipping {path} because it is not a list at the top level")
        return []
    except json.JSONDecodeError:
        print(f"Skipping {path} because it is not valid JSON")
        return []

def project(recs):
    out = []
    for r in recs:
        out.append({
            "para_id": r.get("para_id", ""),
            "context": r.get("context", ""),
            "raw_ocr": r.get("raw_ocr", ""),
            "publication_date": r.get("publication_date", "")
        })
    return out

all_recs = []
for p in inputs:
    recs = load_list_or_empty(p)
    print(f"Loaded {len(recs)} records from {p}")
    all_recs.extend(project(recs))

# deduplicate by para_id keeping the first one seen
uniq = {}
for rec in all_recs:
    pid = rec.get("para_id", "")
    if pid and pid not in uniq:
        uniq[pid] = rec

result = list(uniq.values())

with open(output, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(result)} records to {output}")
print(json.dumps(result[:3], indent=2))

Loaded 439302 records from train.json
Loaded 24111 records from validation.json
Loaded 24084 records from test.json
Wrote 131921 records to document_collection.json
[
  {
    "para_id": "New_Hampshire_18070804_1",
    "context": "Aiscellaneous Repository. From the Albany Register, WAR, OR A PROSPECT OF IT, From recent instances of British Outrage. BY: WILLIAM RAY, Author of the contemplated publication, entitled, \u201cHorrors of Slavery, or the American Turf in Tripoli,\u201d VOTARIES of Freedom, arm! The British Lion roars! Legions of Valor, take th\u2019 alarm\u2014; Rash, rush to guard our shores! Behold the horrid deed\u2014 Your brethren gasping lie! Beneath a tyrant\u2019s hand they bleed\u2014 They groan\u2014they faint\u2014they die. Veterans of seventy-six, Awake the slumbering sword;\u2014 Hearts of your murderous foes transfix\u2014 'Tis vengeance gives the word. Remember Lexington, And Bunker\u2019s tragic hill; \u201cThe same who spilt your blood thereon, Your blood again

## You should check that the collection you have matches that of the paper!

# Create the Test Queries Data Structure

We keep the first 10.000 queries due to memory errors in the free colab version.

To be comparable, please keep the top 10.000 queries for evaluation.

In [ ]:
import json
import re
import unicodedata
import string

input_file = "test.json"
output_file = "test_queries.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

def clean_question(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Extract and clean
queries = [
    {
        "query_id": item.get("query_id", ""),
        "question": clean_question(item.get("question", "")),
    }
    for item in data
]

# Sort by query_id (assuming numeric)
queries = sorted(queries, key=lambda x: int(x["query_id"]) if str(x["query_id"]).isdigit() else x["query_id"])

# Keep only the first 10,000
queries = queries[:10000]

# Save new JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

print(f"Saved {len(queries)} entries to {output_file}")
print(json.dumps(queries[:3], indent=2))

Saved 10000 entries to test_queries.json
[
  {
    "query_id": "test_1",
    "question": "How many lots did Thomas Peirce have"
  },
  {
    "query_id": "test_10",
    "question": "Who gave Hamilton the substance of what he had proposed on the part of General Hamilton"
  },
  {
    "query_id": "test_100",
    "question": "Who informs his FRIENDS and the PUBLIC that he has taken that justly celebrated INN in this city"
  }
]


# Create the Qrels for the test set

In [ ]:
input_file = "test.json"
qrels_file = "test_qrels.json"
answers_file = "test_query_answers.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Build the qrels file: query_id, iteration=0, para_id, relevance=1
qrels = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1
    }
    for item in data
]

# Build the query_answers file: same plus answer and org_answer
query_answers = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1,
        "answer": item.get("answer", ""),
        "org_answer": item.get("org_answer", "")
    }
    for item in data
]

# Save both files
with open(qrels_file, "w", encoding="utf-8") as f:
    json.dump(qrels, f, ensure_ascii=False, indent=2)

with open(answers_file, "w", encoding="utf-8") as f:
    json.dump(query_answers, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qrels)} entries to {qrels_file}")
print(f"Saved {len(query_answers)} entries to {answers_file}")
print("Sample qrels entry:", qrels[0])
print("Sample query_answers entry:", query_answers[0])

Saved 24084 entries to test_qrels.json
Saved 24084 entries to test_query_answers.json
Sample qrels entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1}
Sample query_answers entry: {'query_id': 'test_1', 'iteration': 0, 'para_id': 'New_Hampshire_18030125_16', 'relevance': 1, 'answer': '183', 'org_answer': '183'}


# Retrieval - Good Luck! (YOUR WORK STARTS HERE !)

# Create the Index of the collection (2 min)



In [ ]:
#path index folder
folder_name = "Index"
index_file_name = "terrier_inverted_index"
index_path = os.path.abspath(os.path.join(folder_name, index_file_name))

#init pyTer
if not pt.java.started():
    pt.java.init()

def get_index():
    #check if the index exsists
    properties_file = os.path.join(index_path, "data.properties")
    # Forcing a rebuild by removing the index directory if it exists to ensure updated schema.
    if os.path.exists(index_path):
        print(f"Existing index found at {index_path}. Removing to ensure rebuild with updated schema.")
        shutil.rmtree(index_path)

    print("Index is not found or is being rebuilt, creating a new Index")
    os.makedirs(index_path, exist_ok=True)

    with open('document_collection.json', 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    # Build Data Frame to include docno, context, raw_ocr, and publication_date
    df = pd.DataFrame(raw_data)
    df = df.rename(columns={"para_id": "docno"})[["docno", "context", "raw_ocr", "publication_date"]]

    df['text'] = df['context'] # Create a 'text' column for the primary indexed field

    # Build the index
    indexer = pt.IterDictIndexer(
        index_path,
        meta={"docno": 24, "context": 2048, "raw_ocr": 2048, "text": 2048},
        text_attrs=["text"],           # 'text' (our 'context') is the primary field for indexing content
        meta_reverse=["docno"],        # enable reverse lookup on docno
        pretokenised=False,
        fields=False,
        threads=1,
    )
    index_ref = indexer.index(df.to_dict(orient="records"))
    return pt.IndexFactory.of(index_ref)
index = get_index()

# Print a simple summary
print("Index location:", index_path)
print("Indexed documents:", index.getCollectionStatistics().getNumberOfDocuments())

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-assemblies/5.11/terrier-assemblies-5.11-jar-with-dependenci…

Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...


https://repo1.maven.org/maven2/org/terrier/terrier-python-helper/0.0.8/terrier-python-helper-0.0.8.jar:   0%| …

Done


Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


Index is not found or is being rebuilt, creating a new Index
11:05:04.609 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (Michigan_18370308_24) - further warnings are suppressed
11:05:11.747 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Indexed 1 empty documents
Index location: /content/Index/terrier_inverted_index
Indexed documents: 131921


In [ ]:
# Retrieve collection statistics
stats = index.getCollectionStatistics()

print("Terrier Collection Statistics")
print("--------------------------------")
print(f"Indexed documents:        {stats.getNumberOfDocuments()}")
print(f"Unique terms (vocabulary): {stats.getNumberOfUniqueTerms()}")
print(f"Total tokens:             {stats.getNumberOfTokens()}")
print(f"Average document length:  {stats.getAverageDocumentLength():.2f}")

Terrier Collection Statistics
--------------------------------
Indexed documents:        131921
Unique terms (vocabulary): 236646
Total tokens:             15575099
Average document length:  118.06


# Set up query expansion with histwords (2 mins)

In [ ]:
!wget http://snap.stanford.edu/historical_embeddings/eng-fiction-all_sgns.zip
!unzip -n eng-fiction-all_sgns.zip
!ls -F

--2026-01-11 11:05:12--  http://snap.stanford.edu/historical_embeddings/eng-fiction-all_sgns.zip
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400120997 (382M) [application/zip]
Saving to: ‘eng-fiction-all_sgns.zip’

eng-fiction-all_sgn 100%[===================>] 381.58M  16.2MB/s    in 27s     

2026-01-11 11:05:39 (14.0 MB/s) - ‘eng-fiction-all_sgns.zip’ saved [400120997/400120997]

Archive:  eng-fiction-all_sgns.zip
   creating: sgns/
  inflating: sgns/1990-vocab.pkl     
  inflating: sgns/1970-w.npy         
  inflating: sgns/1810-w.npy         
  inflating: sgns/1840-vocab.pkl     
  inflating: sgns/1920-vocab.pkl     
  inflating: sgns/1950-w.npy         
  inflating: sgns/1980-w.npy         
  inflating: sgns/1950-vocab.pkl     
  inflating: sgns/1830-w.npy         
  inflating: sgns/1830-vocab.pkl     
  inflating: sgns/1880

In [ ]:
def load_histwords_colab(year):
    path_vocab = f"sgns/{year}-vocab.pkl"
    path_vectors = f"sgns/{year}-w.npy"

    print(f"Loading {year} data...")

    with open(path_vocab, 'rb') as f:
        vocab = pickle.load(f)

    vectors = np.load(path_vectors)

    kv = KeyedVectors(vector_size=vectors.shape[1])
    kv.add_vectors(vocab, vectors)

    print(f"Success! Model for {year} loaded with {len(vocab)} words.")
    return kv

hist_model = load_histwords_colab(1850)

Loading 1850 data...
Success! Model for 1850 loaded with 100000 words.


In [ ]:
word = "gay"
if word in hist_model:
    print(f"Synonyms in 1850 for '{word}':")
    print(hist_model.most_similar(word, topn=5))

Synonyms in 1850 for 'gay':
[('brilliant', 0.7579882144927979), ('lively', 0.7548628449440002), ('graceful', 0.6951406598091125), ('cheerful', 0.6921846866607666), ('elegant', 0.6730675101280212)]


/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


### Setup the pipeline before 1st stage retrieval (Query expansion -> Query reduction -> Weighting)

In [ ]:
def hist_expansion_logic(row):
    terms = row['query'].split()
    expanded_query = list(terms)
    seen = {t.split('^')[0].lower() for t in terms}

    penalty = 0.20

    for term in terms:
        parts = term.split('^')
        word = parts[0].lower()
        weight = float(parts[1]) if len(parts) > 1 else 1.0

        if word in hist_model.key_to_index:
            try:
                sim_word, _ = hist_model.most_similar(word, topn=1)[0]
                sim_low = sim_word.lower()

                if sim_low not in seen:
                    expanded_query.append(f"{sim_low}^{round(weight * penalty, 2)}")
                    seen.add(sim_low)
            except:
                continue

    return " ".join(expanded_query)

hist_query_expansion = pt.apply.query(hist_expansion_logic)

In [ ]:
def query_reduction_by_weight(row):
    original_query = row['query']
    # split terms (ex: ["thomas^5.4", "did", "lots^3.2"])
    terms = original_query.split()
    weight_threshold = 4.5 if len(terms) > 3 else 2.0
    reduced_terms = []

    for term in terms:
        if '^' in term:
            word, weight = term.split('^')
            if float(weight) >= weight_threshold:
                reduced_terms.append(term)
        else:
            if term.lower() not in stop_words:
                reduced_terms.append(term)
    return " ".join(reduced_terms) if reduced_terms else original_query

query_reduction = pt.apply.query(query_reduction_by_weight)

example_query = "How many lots did Thomas Peirce have in the year 1800"
reduced_query = query_reduction.transform(pd.DataFrame([{'qid': 'test_q', 'query': example_query}]))

print(f"Original request: {example_query}")
print(f"Reduced request: {reduced_query['query'].iloc[0]}")

Original request: How many lots did Thomas Peirce have in the year 1800
Reduced request: many lots Thomas Peirce year 1800


In [ ]:
import spacy
import math
nlp = spacy.load("en_core_web_sm", disable=["parser"])

def process_query(query_question, index):
    doc = nlp(query_question)
    terms_to_check = [
        token.text for token in doc
        if not token.is_stop and not token.is_punct
    ]

    lex = index.getLexicon()
    stats = index.getCollectionStatistics()
    N = stats.getNumberOfDocuments()

    tokens_with_scores = []
    entities = list(set([ent.text for ent in doc.ents]))

    for word in terms_to_check:
        word_clean = word.lower()
        entry = lex.getLexiconEntry(word_clean)

        if entry:
            df = entry.getDocumentFrequency()
            idf = math.log2(N / df)
            tokens_with_scores.append((word, idf))
        else:
            tokens_with_scores.append((word, 1.0))
    tokens_with_scores.sort(key=lambda x: x[1], reverse=True)

    return tokens_with_scores, entities
def query_weighting_logic(row):
    original_query = row['query'].lower().replace('?', '')

    tokens_scores, entities = process_query(original_query, index)
    score_map = {term.lower(): score for term, score in tokens_scores}
    # print("debug : ", score_map),
    final_terms = []
    threshold = 3.0
    for word in original_query.split():
        score = score_map.get(word)

        if score and score > threshold:
            final_terms.append(f"{word}^{round(score, 2)}")
        else:
            final_terms.append(word)

    return " ".join(final_terms) if final_terms else original_query

query_weighting = pt.apply.query(query_weighting_logic)


## Defining the query pipeline with a test

In [ ]:
# DEFINE IT TO USE IT FOR OUR CUSTOM IR SYSTEM
query_pipeline =  query_weighting  >>  hist_query_expansion >> query_reduction


test_query = "Who lives in Europe with Alex and what is an apple ?"
df_test = pd.DataFrame([["q1", test_query]], columns=['qid', 'query'])
query_weighting_pipeline = pt.apply.query(query_weighting_logic)
transformed_df = query_pipeline.transform(df_test)

print("INPUT:", test_query)
print("OUTPUT:", transformed_df.iloc[0]['query'])

INPUT: Who lives in Europe with Alex and what is an apple ?
OUTPUT: lives europe alex^7.89 apple


/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:449: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]


# 1st stage retrieval

In [ ]:
# First stage retriver with BM25
bm25_1st_stage_retrieval = pt.terrier.Retriever(index, wmodel="BM25")%10
tf_idf_1st_stage_retrieval = pt.terrier.Retriever(index, wmodel="TF_IDF")%10
bm25_custom_1st_stage_retrieval = query_pipeline >> bm25_1st_stage_retrieval

In [ ]:
#testing the engine
query = "Europe"
bm25_results = bm25_1st_stage_retrieval.search(query)
tf_idf_results = tf_idf_1st_stage_retrieval.search(query)
bm25_custom_results = bm25_custom_1st_stage_retrieval.search(query)
#show first 10 results
print('-------------------- bm25 results (top 10) --------------------\n',bm25_results.head(10))
print('-------------------- tf_idf results (top 10) --------------------\n',tf_idf_results.head(10))
print('-------------------- bm25_custom results (top 10) --------------------\n',bm25_custom_results.head(10))

# Convert queries and qrels to pandas DataFrames
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

# Rename columns to match PyTerrier's expected names for evaluation
queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

# WARNING taking the whole queries will take a long time
# --- Use a subset of queries for faster computation,  ---
num_queries_subset = 30 # You can adjust this number

queries_subset_df = queries_df.head(num_queries_subset)
qrels_subset_df = qrels_df[qrels_df['qid'].isin(queries_subset_df['qid'])].copy()

experiment_results = pt.Experiment(
    [bm25_1st_stage_retrieval, tf_idf_1st_stage_retrieval, bm25_custom_1st_stage_retrieval],
    queries_subset_df, # Use the subset of queries
    qrels_subset_df,   # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25","TF-IDF","Custom BM25"]
)

display(experiment_results)

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


-------------------- bm25 results (top 10) --------------------
   qid   docid                     docno  rank      score   query
0   1  122024      New_York_19190610_27     0  10.233243  Europe
1   1   61883         Texas_18771215_13     1   9.759719  Europe
2   1    9637      Maryland_18401110_20     2   9.730148  Europe
3   1  110750      Nebraska_19020225_26     3   9.715430  Europe
4   1   36539  Rhode_Island_18511030_25     4   9.642500  Europe
5   1   47808      Illinois_18680907_40     5   9.578485  Europe
6   1  100207         Kansas_18970902_5     6   9.570657  Europe
7   1   65964     Tennessee_18740915_14     7   9.556417  Europe
8   1   29371        Hawaii_18520522_33     8   9.509314  Europe
9   1   38017     Tennessee_18590726_13     9   9.499877  Europe
-------------------- tf_idf results (top 10) --------------------
   qid   docid                     docno  rank     score   query
0   1  122024      New_York_19190610_27     0  5.641964  Europe
1   1   61883         Tex

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:449: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]


,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25,0.433426,0.366667,0.106667,0.066667,0.533333,0.666667,0.443050,0.487347
1,TF-IDF,0.437354,0.366667,0.100000,0.063333,0.500000,0.633333,0.438290,0.483183
2,Custom BM25,0.345926,0.333333,0.066667,0.043333,0.333333,0.433333,0.333333,0.364877


In [ ]:
import pyterrier as pt

# Let's take the BM25 results for the query "Europe" as an example
# (Assuming bm25_results DataFrame is available from earlier execution)

# Retrieve the 'context' (cleaned text) for the top results
results_with_context = pt.text.get_text(index, 'context')(bm25_results)
print("\n--- BM25 Results with 'context' (cleaned text) ---\n")
display(results_with_context[['docno', 'score', 'context']].head())

# Retrieve the 'raw_ocr' for the top results
results_with_raw_ocr = pt.text.get_text(index, 'raw_ocr')(bm25_results)
print("\n--- BM25 Results with 'raw_ocr' ---\n")
display(results_with_raw_ocr[['docno', 'score', 'raw_ocr']].head())


--- BM25 Results with 'context' (cleaned text) ---



,docno,score,context
0,New_York_19190610_27,10.233243,It should be a little full and extend about ei...
1,Texas_18771215_13,9.759719,When further we reflect that Russia is Slavoni...
2,Maryland_18401110_20,9.730148,In the south it has almost accomplished its tr...
3,Nebraska_19020225_26,9.715430,"St. Louis, via Southampton (mail for Ireland m..."
4,Rhode_Island_18511030_25,9.642500,"They said they pitied us, honored our efforts,..."



--- BM25 Results with 'raw_ocr' ---



,docno,score,raw_ocr
0,New_York_19190610_27,10.233243,It It be a little full and extend\nif nVinut e...
1,Texas_18771215_13,9.759719,When\nfurther we reflect that Russia is Scla\n...
2,Maryland_18401110_20,9.730148,"In"" the\nsouth it has almost accomplished its ..."
3,Nebraska_19020225_26,9.715430,"St. Louis, via Southampton (mall\nor Ireland m..."
4,Rhode_Island_18511030_25,9.642500,"They eaid they pit\nied ug, honored our eflort..."


# RUN THIS CODE IF GPU AND TIME AVAILABLE

MESSAGE: THE CODE BELOW IS SUPPOSED TO RETRIEVE THE WHOLE QUERIES BUT BECAUSE OF COMPUTATION SPEED, IT DIDN'T GIVE A RESULT IN A REASONABLE TIME

In [ ]:
"""
experiment_results = pt.Experiment(
    [bm25_1st_stage_retrieval, tf_idf_1st_stage_retrieval, bm25_custom_1st_stage_retrieval],
    queries_df, # Use all the queries
    qrels_df,   # Use all the qrels
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25","TF-IDF","BM25 Query Expansion histwords"]
)
"""

'\nexperiment_results = pt.Experiment(\n    [bm25_1st_stage_retrieval, tf_idf_1st_stage_retrieval, bm25_hist_qe_1st_stage_retrieval],\n    queries_df, # Use all the queries\n    qrels_df,   # Use all the qrels\n    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],\n    names=["BM25","TF-IDF","BM25 Query Expansion histwords"]\n)\n'

DISPLAY and SAVE the results to avoid 22 minutes of waiting

In [ ]:
display(experiment_results)
experiment_results.to_csv("experiment_results_all_queries.csv", index=False) # DON'T FORGET TO DOWNLOAD IT

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25,0.571993,0.52,0.128,0.071,0.64,0.71,0.578709,0.602405
1,TF-IDF,0.576489,0.52,0.126,0.070,0.63,0.70,0.579284,0.602801
2,BM25 Query Expansion histwords,0.379656,0.32,0.092,0.049,0.46,0.49,0.392204,0.401811


# 2nd stage retrieval

In [ ]:
# Initialize MonoT5ReRanker
monoT5_reranker = MonoT5ReRanker(batch_size=8)
monoT5_cleaned_reranker = (
    query_pipeline
    >> bm25_1st_stage_retrieval
    >> pt.text.get_text(index, "context")
    >> pt.apply.generic(lambda df: df.rename(columns={'context': 'text'}))
    >> monoT5_reranker
)

# Correction pour l'OCR brut
monoT5_raw_ocr_reranker = (
    query_pipeline
    >> bm25_1st_stage_retrieval
    >> pt.text.get_text(index, "raw_ocr")
    >> pt.apply.generic(lambda df: df.rename(columns={'raw_ocr': 'text'}))
    >> monoT5_reranker
)

query = "Europe"
monoT5_cleaned_results = monoT5_cleaned_reranker.search(query)
monoT5_raw_ocr_results = monoT5_raw_ocr_reranker.search(query)
print('-------------------- Cleaned MonoT5 Re-ranking results (top 10) --------------------\n', monoT5_cleaned_results.head(10))
print('-------------------- OCR MonoT5 Re-ranking results (top 10) --------------------\n', monoT5_raw_ocr_results.head(10))

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


monoT5:   0%|          | 0/2 [00:00<?, ?batches/s]

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


monoT5:   0%|          | 0/2 [00:00<?, ?batches/s]

-------------------- Cleaned MonoT5 Re-ranking results (top 10) --------------------
   qid   docid                     docno query_2 query_1               query_0  \
2   1    9637      Maryland_18401110_20  Europe  europe  europe continent^0.2   
9   1   38017     Tennessee_18590726_13  Europe  europe  europe continent^0.2   
3   1  110750      Nebraska_19020225_26  Europe  europe  europe continent^0.2   
4   1   36539  Rhode_Island_18511030_25  Europe  europe  europe continent^0.2   
8   1   29371        Hawaii_18520522_33  Europe  europe  europe continent^0.2   
0   1  122024      New_York_19190610_27  Europe  europe  europe continent^0.2   
5   1   47808      Illinois_18680907_40  Europe  europe  europe continent^0.2   
1   1   61883         Texas_18771215_13  Europe  europe  europe continent^0.2   
6   1  100207         Kansas_18970902_5  Europe  europe  europe continent^0.2   
7   1   65964     Tennessee_18740915_14  Europe  europe  europe continent^0.2   

    query             

In [ ]:
# Prepare queries_df and qrels_df as before (assuming 'queries' and 'qrels' are defined globally)
queries_df = pd.DataFrame(queries)
qrels_df = pd.DataFrame(qrels)

queries_df = queries_df.rename(columns={"query_id": "qid", "question": "query"})
qrels_df = qrels_df.rename(columns={"query_id": "qid", "para_id": "docno"})

num_queries_subset = 30

queries_subset_df = queries_df.head(num_queries_subset)
qrels_subset_df = qrels_df[qrels_df['qid'].isin(queries_subset_df['qid'])].copy()

experiment_results_monoT5 = pt.Experiment(
    [monoT5_cleaned_reranker,monoT5_raw_ocr_reranker],
    queries_subset_df, # Use the subset of queries
    qrels_subset_df,   # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25_MonoT5_Reranked_Cleaned_Context", "BM25_MonoT5_Reranked_RawOCR"]
)

display(experiment_results_monoT5)

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:449: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]


monoT5:   0%|          | 0/38 [00:00<?, ?batches/s]

/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/usr/local/lib/python3.12/dist-packages/gensim/models/keyedvectors.py:449: RuntimeWarning: invalid value encountered in divide
  result = self.vectors[index] / self.norms[index]


monoT5:   0%|          | 0/38 [00:00<?, ?batches/s]

,name,AP,P@1,P@5,P@10,R@5,R@10,nDCG@5,nDCG@10
0,BM25_MonoT5_Reranked_Cleaned_Context,0.388889,0.366667,0.086667,0.043333,0.433333,0.433333,0.400000,0.40000
1,BM25_MonoT5_Reranked_RawOCR,0.302222,0.233333,0.080000,0.043333,0.400000,0.433333,0.322216,0.33409


# RUN THIS CODE IF GPU AND TIME AVAILABLE !

In [ ]:
"""experiment_results_monoT5 = pt.Experiment(
    [monoT5_cleaned_reranker,monoT5_raw_ocr_reranker],
    queries_subset_df, # Use the subset of queries
    qrels_subset_df,   # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25_MonoT5_Reranked_Cleaned_Context", "BM25_MonoT5_Reranked_RawOCR"]
)

display(experiment_results_monoT5_E1)
experiment_results_monoT5_E1.to_csv("experiment_results_all_queries.csv", index=False)"""

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

def unsloth_gemma_scorer(row):
    query = row['query']
    text = row['text'][:600]

    messages = [
        {"role": "user", "content": f"Is this document relevant to the query? Answer only with a score from 0 to 10.\nQuery: {query}\nDocument: {text}\nScore:"},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 5)
    decoded_output = tokenizer.batch_decode(outputs[:, inputs.shape[1]:], skip_special_tokens=True)[0]

    import re
    numbers = re.findall(r'\d+', decoded_output)
    return float(numbers[0]) if numbers else 0.0



In [ ]:
def logit_discriminator_scorer(row):
    query = str(row['query'])
    text = str(row['text'])[:400]
    prompt = f"Query: {query} Document: {text} Relevant? (0-10):"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(input_ids=inputs.input_ids)
        last_token_logits = outputs.logits[0, -1, :]
        # Encode digits 0-10 to find their token IDs
        target_tokens = [tokenizer.encode(str(i), add_special_tokens=False)[-1] for i in range(11)]

        # Filter logits for only the target digit tokens
        relevant_logits = last_token_logits[target_tokens]

        # Apply softmax to get probabilities
        probs = F.softmax(relevant_logits, dim=-1)

        # Calculate expected score (0*P(0) + 1*P(1) + ... + 10*P(10))
        final_score = sum(i * prob.item() for i, prob in enumerate(probs))

    return final_score

sample_query = queries_df.iloc[0]['query'] # Example: "How many lots did Thomas Peirce have"

# Assuming 'result' (from document_collection.json) is available and contains documents
# We'll create a small DataFrame with a single query and a few document contexts to test.
sample_data_for_scorer = [
    {"query": sample_query, "text": result[0]['context']},
    {"query": sample_query, "text": result[1]['context']},
    {"query": "European politics", "text": result[2]['context']}
]
sample_df = pd.DataFrame(sample_data_for_scorer)
sample_df['refined_score'] = sample_df.apply(logit_discriminator_scorer, axis=1)
print(sample_df[['query', 'refined_score']])

llm_reranker_cleaned = (
    query_pipeline
    >> bm25_1st_stage_retrieval
    >> pt.text.get_text(index, "context")
    >> pt.apply.generic(lambda df: df.rename(columns={'context': 'text'}))
    >> pt.apply.doc_scores(logit_discriminator_scorer)
)
llm_reranker_ocr_text = (
    query_pipeline
    >> bm25_1st_stage_retrieval
    >> pt.text.get_text(index, "raw_ocr")
    >> pt.apply.generic(lambda df: df.rename(columns={"raw_ocr": 'text'}))
    >> pt.apply.doc_scores(logit_discriminator_scorer)
)
query_llm_test = "Europe"
llm_reranked_results1 = llm_reranker_cleaned.search(query_llm_test)
llm_reranked_results2 = llm_reranker_ocr_text.search(query_llm_test)

print('\n-------------------- LLM Re-ranking results (top 10) --------------------\n', llm_reranked_results1.head(10))
print('\n-------------------- LLM Re-ranking results (top 10) --------------------\n', llm_reranked_results2.head(10))

# Prepare queries_df and qrels_df (assuming 'queries' and 'qrels' are defined globally)
queries_df_llm = pd.DataFrame(queries)
qrels_df_llm = pd.DataFrame(qrels)

queries_df_llm = queries_df_llm.rename(columns={"query_id": "qid", "question": "query"})
qrels_df_llm = qrels_df_llm.rename(columns={"query_id": "qid", "para_id": "docno"})

# Use a small subset of queries for faster computation
num_queries_subset_llm = 30
queries_subset_df_llm = queries_df_llm.head(num_queries_subset_llm)
qrels_subset_df_llm = qrels_df_llm[qrels_df_llm['qid'].isin(queries_subset_df_llm['qid'])].copy()

# Run the experiment with the LLM re-ranking pipeline
experiment_results_llm = pt.Experiment(
    [llm_reranker_cleaned],
    queries_subset_df_llm,  # Use the subset of queries
    qrels_subset_df_llm,    # Use the filtered qrels for the subset
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP], # Ensure these measures are defined
    names=["BM25_LLM_Reranked"]
)

display(experiment_results_llm)


# RUN THIS CODE IF GPU AND TIME AVAILABLE

In [ ]:
experiment_results_reranked_E1 = pt.Experiment(
    [llm_reranker_cleaned,llm_reranker_ocr_text],
    queries_df,
    qrels_df,
    [P@1, P@5, P@10, R@5, R@10, nDCG@5, nDCG@10, MAP],
    names=["BM25_LLM_Reranked"]
)

display(experiment_results_reranked_E1)

### Using ASPIRE (TEMPORARY)


PyTerrier : https://pyterrier.readthedocs.io/en/latest/experiments.html

ASPIRE https://github.com/GiorgosPeikos/ASPIRE

ONLINE: https://aspire-ir-eval.streamlit.app/



Remember the res files are actually the rankings created by our retrieval model!

We downloading it to use another tool for data analysis. The tool is called ASPIRE.

ASPIRE requires a different input format!

In [ ]:
# ----- RESULTS (TF-IDF, BM25, etc.) -----
def save_res(df, run_name, out_path, sep="\t"):
    res = df.copy()  # keep original untouched
    if "docid" in res.columns and "docno" not in res.columns:
        res = res.rename(columns={"docid": "docno"})  # normalize doc id
    # The 'query' column is often added by pyterrier for inspection but not needed in output
    res = res.drop(columns=["query", "query_0"], errors="ignore") # remove query text, keep qid
    res["iteration"] = "Q0"  # fixed iteration value
    res["experiment_id"] = run_name  # label for the run
    res = res.rename(columns={"qid": "query_id", "docno": "doc_id"})  # final names
    # Ensure score is numeric for sorting, if needed, though pyterrier usually keeps it
    res["score"] = pd.to_numeric(res["score"])
    # Sort by score for ranking within each query_id
    res = res.sort_values(by=["query_id", "score"], ascending=[True, False])
    res["rank"] = res.groupby("query_id")["score"].rank(ascending=False, method="first").astype(int)
    res = res[["query_id", "iteration", "doc_id", "rank", "score", "experiment_id"]]  # correct column order
    res.to_csv(out_path, sep=sep, index=False, header=False)  # write file


# Generate full retrieval results for all queries
# Assuming 'bm25', 'tf_idf', 'bm25_hist_qe', 'queries_df' are available from previous cells
full_bm25_results = bm25.transform(queries_df)
full_tf_idf_results = tf_idf.transform(queries_df)
full_bm25_hist_qe_results = bm25_hist_qe.transform(queries_df)

# Save the results to CSV files for ASPIRE
save_res(full_bm25_results, "BM25", "results_bm25.csv")
save_res(full_tf_idf_results, "TF-IDF", "results_tf_idf.csv")
save_res(full_bm25_hist_qe_results, "BM25_HistQE", "results_bm25_hist_qe.csv")

print("Generated results CSVs for BM25, TF-IDF, and BM25 with historical QE.")

# ----- QUERIES -----
# root with task attribute
# Renaming 'qid' to 'query_id' for consistency if needed, but XML expects 'number' attribute
root = ET.Element("topics", attrib={"task": "Chronicling America QA"})  # create root element

# add one <topic number=\"...\"> per row
for _, row in queries_df.iterrows():  # iterate over dataframe rows (using queries_df)
    topic = ET.SubElement(root, "topic", attrib={"number": str(row["qid"])})  # topic node
    topic.text = str(row["query"])  # topic content

# pretty print
rough_xml = ET.tostring(root, encoding="utf-8")  # raw bytes
pretty_xml = minidom.parseString(rough_xml).toprettyxml(indent="  ", encoding="utf-8")  # formatted bytes

# write to file
with open("queries.xml", "wb") as f:  # save xml with declaration
    f.write(pretty_xml)

print("Generated queries.xml file.")

# ----- QRELS -----
qrels_out = qrels_df.copy() # Use qrels_df which is available in kernel state
qrels_out = qrels_out.drop(columns=["iteration"], errors="ignore") # iteration from qrels_df is int, ASPIRE wants string 'Q0'
if "docid" in qrels_out.columns and "docno" not in qrels_out.columns:
    qrels_out = qrels_out.rename(columns={"docid": "docno"})
qrels_out = qrels_out.rename(columns={"qid": "query_id", "docno": "doc_id"})
qrels_out["iteration"] = "0" # Ensure '0' string for ASPIRE format
qrels_out = qrels_out[["query_id", "iteration", "doc_id", "relevance"]]
qrels_out.to_csv("qrels.csv", sep=" ", index=False, header=False)

print("Generated qrels.csv file.")

print("All files generated for ASPIRE compatibility.")

In [ ]:
# To use ASPIRE, we need to download the res files, the qrels and queries in the
# required dataformat and use the interface or use it locally.

# Res file Expected format: query_id, iteration (i.e. Q0), doc_id, rank, score, experiment_id
# Queries Expected format: query_id, query
# Qrels Expected format: query_id, iteration (i.e. Q0), doc_id, relevance (without header row)

# DEMO TIME!